In [1]:
import pandas as pd
from google.colab import auth
from google.colab import files
from tqdm import tqdm_notebook
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
with open("/content/cities.txt","r") as f:
    lines = f.readlines()

cities = [line.strip() for line in lines]
subreddits = ['nyc', 'LosAngeles', 'chicago', 'Dallas', 'houston', 'washingtondc', 'Miami', 'philadelphia', 'Atlanta', 'phoenix', 'boston', 'sanfrancisco', 'Riverside', 'Detroit', 'Seattle', 'Minneapolis', 'sandiego', 'tampa', 'Denver', 'StLouis', 'baltimore', 'Charlotte', 'orlando', 'sanantonio', 'Portland', 'Sacramento', 'pittsburgh', 'LasVegas', 'Austin', 'cincinnati', 'kansascity', 'Columbus', 'indianapolis', 'Cleveland', 'SanJose', 'nashville', 'VirginiaBeach', 'providence', 'milwaukee', 'jacksonville', 'oklahomacity', 'raleigh', 'richmond', 'NewOrleans', 'Louisville', 'SaltLakeCity', 'Hartford', 'Buffalo', 'Birmingham']

dates=[]
for i in range(1,13):
    for year in range(5,9):
        month = '0{}'.format(i)[-2:]
        dates.append('201{}_{}'.format(year, month))

In [3]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='socialseg')

for city_idx in tqdm_notebook(range(len(cities))):
    bodies = []
    subrs = []
    authors = []

    for date in dates:
        query = """
                SELECT body,subreddit,author 
                FROM `fh-bigquery.reddit_comments.{}` 
                WHERE (body LIKE @string OR body LIKE @string2) AND (subreddit = @subreddit)
                """.format(date)

        job_config = bigquery.QueryJobConfig(
            query_parameters=[bigquery.ScalarQueryParameter("string", "STRING", '%{}%'.format(cities[city_idx])),
                              bigquery.ScalarQueryParameter("string2", "STRING", '%{}%'.format(cities[city_idx].lower())),
                              bigquery.ScalarQueryParameter("subreddit", "STRING", subreddits[city_idx]),])
        query_job = client.query(query, job_config=job_config)  # Make an API request.

        for row in query_job:
            bodies.append(row.body)
            subrs.append(row.subreddit)
            authors.append(row.author)
    
    print("Collect {} comments from r/{}".format(len(bodies),subreddits[city_idx]))
    df = pd.DataFrame({'body':bodies,'subreddit':subrs,'author':authors})
    df.to_csv('{}.csv'.format(cities[city_idx]),header=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Collect 70008 comments from r/nyc
Collect 20377 comments from r/LosAngeles
Collect 152366 comments from r/chicago
Collect 44640 comments from r/Dallas
Collect 118987 comments from r/houston
Collect 16062 comments from r/washingtondc
Collect 23825 comments from r/Miami
Collect 30910 comments from r/philadelphia
Collect 88358 comments from r/Atlanta
Collect 27972 comments from r/phoenix
Collect 107217 comments from r/boston
Collect 22771 comments from r/sanfrancisco
Collect 170 comments from r/Riverside
Collect 37089 comments from r/Detroit
Collect 123094 comments from r/Seattle
Collect 16676 comments from r/Minneapolis
Collect 31631 comments from r/sandiego
Collect 22466 comments from r/tampa
Collect 71241 comments from r/Denver
Collect 19289 comments from r/StLouis
Collect 51090 comments from r/baltimore
Collect 29810 comments from r/Charlotte
Collect 21643 comments from r/orlando
Collect 10794 comments from r/sanantonio
Collect 140681 comments from r/Portland
Collect 21116 comments fr